In [1]:
import sys
import numpy as np
import pandas as pd

sys.path.append("../../../../")

from src.ab.design import ClassicABDesign, SequentialABDesign, BayesianABDesign
from src.ab.experiment import SingleExperiment, ExperimentSeries
from src.ab.data import BernoulliDataGenerator

#import AB parameters (confidence level, power, base conversion rate value, etc) from config
from config import alpha, beta, power, GLOBAL_SIZE_TH, base_ctr, \
                   mde, update_size, N_experiments, n_groups, \
                   alpha_prior, beta_prior, resolution, toc, iterations, prob_th, \
                   ab_type

#define prefix (I_TYPE_ERROR or II_TYPE_ERROR)
prefix = 'I_TYPE_ERROR'
#string will be added to an output report filename
params = 'toc_{}_a_{}_b_{}'.format(toc,
                                   alpha_prior,
                                   beta_prior)

In [2]:
#create a pandas dataframe for a final report
total_stats_df = pd.DataFrame()
#base_ctrs array
ctrs = []
#i_type_errors array
itype_errors = []
#avg sample array
size_avg = []
#total sample size array
size_total = []
#std array
size_std = []

In [3]:
#run series of experiments per every base conversion rate
for base_ctr in np.arange(0.65, 0.67, 0.01):
    print(base_ctr)
    exp_series = ExperimentSeries(N_experiments=N_experiments,
                                  n_groups=n_groups,
                                  base_ctr=base_ctr,
                                  true_ctrs=[base_ctr,
                                             base_ctr],
                                  ab_type=ab_type,
                                  alpha=alpha,
                                  power=power,
                                  mde=mde,
                                  update_size=update_size,
                                  alpha_prior=alpha_prior,
                                  beta_prior=beta_prior,
                                  resolution=resolution,
                                  toc=toc,
                                  iterations=iterations)
    exp_series.run_series()
    I_type_error = np.sum(exp_series.winners) / float(len(exp_series.winners))
    sample_mean = np.mean(exp_series.sample_sizes)
    print(np.sum(exp_series.sample_sizes))
    sample_sum = np.sum(exp_series.sample_sizes)
    sample_std = np.std(exp_series.sample_sizes)
    ctrs.append(base_ctr)
    itype_errors.append(I_type_error)
    size_avg.append(sample_mean)
    size_total.append(sample_sum)
    size_std.append(sample_std)
    print("I type error: {}".format(I_type_error))

    current_ctr_stats_df = pd.DataFrame({'sample_size': exp_series.sample_sizes})
    current_ctr_stats_df['ctr'] = base_ctr
    current_ctr_stats_df['ab_type'] = ab_type

0.65
8619600
I type error: 0.004
0.66
8489700
I type error: 0.0
0.67
8364200
I type error: 0.001


In [4]:
#create a report
total_stats_df = pd.DataFrame({'base_ctr': ctrs,
                             'i_type_error': itype_errors,
                             'sample_size_total': size_total,
                             'sample_size_avg': size_avg,
                             'sample_size_std': size_std
                             })
total_stats_df['mde'] = mde
total_stats_df['alpha'] = alpha
total_stats_df['beta'] = beta
total_stats_df

,base_ctr,i_type_error,sample_size_total,sample_size_avg,sample_size_std,mde,alpha,beta
0,0.65,0.004,8619600,8619.6,84.355438,0.05,0.05,0.8
1,0.66,0.000,8489700,8489.7,52.951959,0.05,0.05,0.8
2,0.67,0.001,8364200,8364.2,59.819395,0.05,0.05,0.8


In [5]:
#save final report
total_stats_df[['mde',
                    'alpha',
                    'beta',
                    'base_ctr',
                    'i_type_error',
                    'sample_size_total',
                    'sample_size_avg',
                    'sample_size_std']].round(4).to_csv("reports/{prefix}___{ab_type}_{params}.csv"\
                                                        .format(prefix=prefix,
                                                                ab_type=ab_type,
                                                                params=params),
                                               index=None)